In [ ]:
import sys

sys.path.append("../")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
import numpy as np
from scripts.results import show_yolo_prediction

In [ ]:
sns.set_theme(style="whitegrid", context="talk", palette="Set2")

#### Training Results

In [ ]:
results = pd.read_csv("../models/final_model/results.csv")

In [ ]:
results

In [ ]:
metrics = [
    "metrics/precision(B)",
    "metrics/recall(B)",
    "metrics/mAP50(B)",
]

df = results.melt(
    id_vars="epoch", value_vars=metrics, var_name="metric", value_name="value"
)

plt.figure(figsize=(10, 6))
sns.lineplot(df, x="epoch", y="value", hue="metric")
plt.title("YOLO Accuracy Metrics")
plt.ylabel("Value")
plt.xlabel("Epoch")
plt.legend(title="Metric")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True)

losses = ["box_loss", "cls_loss", "dfl_loss"]

for i, loss in enumerate(losses):
    ax = axes[i]
    train_col = f"train/{loss}"
    val_col = f"val/{loss}"

    sns.lineplot(
        data=results,
        x="epoch",
        y=train_col,
        ax=ax,
        label="train",
        color=sns.color_palette()[i],
    )
    sns.lineplot(
        data=results,
        x="epoch",
        y=val_col,
        ax=ax,
        label="val",
        linestyle="--",
        color=sns.color_palette()[i],
    )

    ax.set_title(loss.replace("_", " ").title())
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    ax.grid(True, linestyle=":", alpha=0.6)

plt.tight_layout()
plt.show()

#### Prediction Results

In [ ]:
show_yolo_prediction(
    model_path="../models/final_model/weights/best.pt",
    root_dir="../data/test",
    split="val",
    conf=0.2,  # change confidence here
)

In [ ]:
model = YOLO("../models/final_model/weights/best.pt")

metrics = model.val(
    data="../data/test/ct.yaml",
    project="../models",
    name="final_model_0.0001",
    exist_ok=True,
    split="val",
    conf=0.0001,  # set low to see how metrics change with confidence
    iou=0.5,
    verbose=False,
)

In [ ]:
df = pd.DataFrame(
    {
        "Confidence": np.linspace(0, 1, 1000),
        "Precision": metrics.box.p_curve[0],
        "Recall": metrics.box.r_curve[0],
        "F1 Score": metrics.box.f1_curve[0],
    }
)

df_long = df.melt("Confidence", var_name="Metric", value_name="Score")

# 3. Plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long, x="Confidence", y="Score", hue="Metric", linewidth=2.5)

plt.title("Confidence vs. Metrics")
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.ylim(0, 1.05)
plt.show()

In [ ]:
metrics = model.val(
    data="../data/test/ct.yaml",
    project="../models",
    name="final_model_0.1",
    exist_ok=True,
    split="val",
    conf=0.1,  # change conf here
    iou=0.5,
    verbose=False,
)

In [ ]:
cm_obj = metrics.confusion_matrix
cm = cm_obj.matrix.T


annot_labels = np.empty_like(cm, dtype=object)
rows, cols = cm.shape

for i in range(rows):
    for j in range(cols):
        val = int(cm[i, j])
        annot_labels[i, j] = f"{val}" if val > 0 else ""

# Plot
plt.figure(figsize=(8, 6))

sns.heatmap(
    cm,
    annot=annot_labels,
    fmt="",
    cmap="Blues",
    cbar=False,
    xticklabels=["Nodule", "Background"],
    yticklabels=["Nodule", "Background"],
    linewidths=1,
    linecolor="black",
)

plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix @ Conf=0.1")
plt.tight_layout()
plt.show()